# Quantum Sudoku Solver

#TODO description of project...?

In [1]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, AncillaRegister, Parameter
from qiskit.quantum_info import Statevector, Operator
import pylatexenc

import matplotlib.pyplot as plt

import numpy as np

## Representing a sudoku puzzle.

For now, we will assume that the input is a $4 \times 4$ sudoku puzzle with a unique solution.

The sudoku puzzle may be initially represented as a $4 \times 4$ matrix or 2D array, but to deal with empty cells, I decided to represent the puzzle (in any state of incompletion) as a dictionary, where the keys are tuples (representing the coordinates of a cell) and the value associated with a key is a set of possible values that the cell can take, absent the rules of sudoku.

### Example

Suppose we are given the following sudoku puzzle.

```
2 0 3 1
1 * * 0
0 * * *
3 * * 2
```

We can think of the puzzle as a matrix $(a_{ij})$. (Let's use 0-based indexing.)
The dictionary representing this puzzle would be written as follows.


In [2]:
puzzle_dict_example = {
    (0,0) : {2},
    (0,1): {0},
    (0,2): {3},
    (0,3): {1},
    (1,0): {1},
    (1,1): {0,1,2,3},
    (1,2): {0,1,2,3},
    (1,3): {0},
    (2,0): {0},
    (2,1): {0,1,2,3},
    (2,2): {0,1,2,3},
    (2,3): {0,1,2,3},
    (3,0): {3},
    (3,1): {0,1,2,3},
    (3,2): {0,1,2,3},
    (3,3): {2}
}

puzzle_dict_example

{(0, 0): {2},
 (0, 1): {0},
 (0, 2): {3},
 (0, 3): {1},
 (1, 0): {1},
 (1, 1): {0, 1, 2, 3},
 (1, 2): {0, 1, 2, 3},
 (1, 3): {0},
 (2, 0): {0},
 (2, 1): {0, 1, 2, 3},
 (2, 2): {0, 1, 2, 3},
 (2, 3): {0, 1, 2, 3},
 (3, 0): {3},
 (3, 1): {0, 1, 2, 3},
 (3, 2): {0, 1, 2, 3},
 (3, 3): {2}}

In the above example, the cell indexed by $(0,0)$ has the value $2$, so in its dictionary representation, the value assigned to the key $(0,0)$ is the set $\{2\}$.
In contrast, the cell indexed by $(1,2)$ is empty, so before applying any sudoku rules, we naively assume that all values are possible. Thus, the value assigned to the key $(1,2)$ is the set $\{0,1,2,3\}$.

## Qubit representation

To solve this sudoku with Grover's algorithm, we need to represent the puzzle in qubits.

In a $4 \times 4$ puzzle, we require two bits to represent the possible values $\{0,1,2,3\}$ in binary, so if the puzzle has $k$ empty cells, then we need $2k$ qubits to represent these $k$ empty cells.

In our dictionary representation of the puzzle, we can detect that a cell is empty when the corresponding value is not a singleton.
We will store the empty cells in an ordered data structure, since we need to keep track of the order of the qubits.

In [5]:
empty_cells_list_example = list()
for key in puzzle_dict_example.keys():
    if len(puzzle_dict_example.get(key)) > 1:
        empty_cells_list_example.append(key)

empty_cells_tuple_example = tuple(empty_cells_list_example)
empty_cells_tuple_example

((1, 1), (1, 2), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2))

In [7]:
#quantum_register_example = QuantumRegister(size=2*len(empty_cells_tuple_example), name="x")

#quantum_circuit_example = QuantumCircuit(quantum_register_example, name=r"example circuit")

#quantum_circuit_example.draw(output="mpl")

In the example puzzle above, there are 7 empty cells, so we need 14 qubits $x_0, x_1, \dots, x_{13}$.
For the $j$-th element of `empty_cells_tuple_example$, the qubit $x_{2j}$ represents the least significant bit of the value that will be assigned to this empty cell, and the qubit $x_{2j+1}$ represents the next significant bit.

For example, consider `empty_cells_tuple_example[2]`, which is the cell indexed by $(2,1)$.
The value that will be assigned to this cell has the binary representation $x_5 x_4$, i.e. the value can be written as $2^1 x_5 + 2^0 x_4$.

# Grover's algorithm for $4 \times 4$ sudoku puzzles

Suppose we are given a $4 \times 4$ sudoku puzzle with a unique solution, represented in the same way as the above example.
We want to construct a quantum circuit that implements Grover's algorithm, where the marker oracle checks whether or not the rules of sudoku are violated.


### Helper functions

#TODO

In [ ]:
# def get_empty(puzzle):
    # TODO
    # return empty_cells

In [ ]:
# def get_rules(puzzle):
    # TODO
    # return rules

In [ ]:
# def get_measurement(quantum_circuit):
    #TODO output distribution
    #TODO most likely outcome
    #TODO convert to bit string as tuple
    # return bin_rep

### Marker oracle

#TODO

In [ ]:
# Marker oracle

# def marker_oracle(empty_cells, rules):
    # quantum_register = 
    # rules_ancilla = 
    # oracle_circuit = 
    # TODO work
    # return oracle_circuit.to_gate()

### Grover diffuser

#TODO

In [ ]:
# Grover diffuser

# def grover_diffuser(n):
    # TODO
    # diffuser_circuit =
    #return diffuser_circuit.to_gate()

### Grover iteration

#TODO

In [ ]:
# Grover iteration

# def grover_iteration(empty_cells, rules):
    # size = 2*len(empty_cells)
    # quantum_register =
    # iteration_circuit =
    # iteration_circuit.compose(marker_oracle(empty_cells, rules), inplace=True)
    # iteration_circuit.compose(grover_diffuser(size), qubits=quantum_register, inplace=True)
    # return iteration_circuit.to_gate()

### Finally, the Grover circuit

#TODO

In [ ]:
# Grover circuit

# def grover(empty_cells, rules):
    # quantum_register = 
    # ancilla register =
    # grover_circuit =
    # grover_circuit.h(quantum_register)
    # n = 2 * len(empty_cells)
    # R = np.ceil(np.pi * sqrt(2**n) / 4)
    # for i in range(R):
        # grover_circuit.compose(grover_iteration(empty_cells, rules),
                               # inplace=True)
    # return grover_circuit


In [ ]:
#TODO look at output distribution

### Putting it all together

#TODO let's finally solve the sudoku

In [ ]:
# def qc_solve(puzzle):
    # empty_cells = get_empty(puzzle)
    # rules = get_rules(puzzle)
    # grover_circuit = grover(empty_cells, rules)
    # bin_rep = qubits_to_binary(grover_circuit) # tuple
    # dec_rep = #TODO convert back to decimal
    # filled_cells = {empty_cells[i]: dec_rep[i] for i in range(len(empty_cells))}
    # puzzle.update(filled_cells)
    # return puzzle

## Testing on random puzzles

#TODO 

In [ ]:
#TODO generate random puzzle

In [ ]:
#TODO test qc_solve